## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données

### **Lire le corpus** 

In [440]:
import shutil, re, pandas, random
from os import listdir, chdir, path
from pathlib import Path

acteur = 'chum'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    base_path = '../03-corpus/2-sous-corpus/'
    file_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else: 
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur) + '.csv'

In [441]:
from pandas import *
with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path)['text'].tolist()
        #data = data[~data["url"].str.contains('pdf')] # Si on veut exclure les PDFs

### **Nettoyage**

In [442]:
text = [t.strip('\n').lower().replace('’', '\'') for t in data]
punct = '[!#$%&\(\)*+,-/:;<=>?@[\]^_{|}~©«»—]'
spaces = '\s+'

text = [re.sub(punct, ' ', t).replace("' ", "'" ) for t in text]
text = [re.sub(spaces, ' ', t) for t in text]

In [443]:
nb_docs = len(text)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 2299 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [444]:
n = round(1 * nb_docs)
corpus = random.sample(text, n)

print("On va travailler sur un échantillon correspondant à environ x % des documents du corpus, soit {} documents". format(len(corpus)))

corpus = " ".join(corpus)

On va travailler sur un échantillon correspondant à environ x % des documents du corpus, soit 2299 documents


**NLTK**\
https://www.nltk.org/ 

In [445]:
import nltk
#nltk.download(['popular'])

### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [446]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [447]:
for mwe in mwe_sw:
    corpus = corpus.replace(mwe, ' MWE_STOP ').replace('  ', " ")

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [449]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

tokens = tokenizer_re.tokenize(corpus)

print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))

Avec le RegExpTokenizer, notre corpus contient 839760 tokens.


In [450]:
temps = round(len(tokens) / 15000 / 60)
print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

Le POS tagging devrait prendre environ 1 minutes.


In [451]:
corpus = " ".join(tokens).replace("' ", "'")

In [452]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [453]:
#file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

#with open(file_path) as f:
#    csv = read_csv(f)

#treeTag = [term for term in csv['TreeTagger'].tolist()] 
#lefff = [term for term in csv['Lefff'].tolist()]

#mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [454]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]
#  mapping[t.split('\t')[1]])
tagged = [(t.split('\t')[0], t.split('\t')[1]) for t in tagger.tag_text(corpus)]


#if len(t.split('\t')) >1

In [455]:
tagged[:25]

[('christiane', 'NOM'),
 ('boulanger', 'ADJ'),
 ('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('chum', 'NOM'),
 ('navigation', 'NOM'),
 ('MWE_STOP', 'NAM'),
 ('patients', 'ADJ'),
 ('répertoire', 'NOM'),
 ('enseignement', 'NOM'),
 ('et', 'KON'),
 ('académie', 'NOM'),
 ('centre', 'NOM'),
 ('de', 'PRP'),
 ('recherche', 'NOM'),
 ('innovation', 'NOM'),
 ('nouvelles', 'ADJ'),
 ('carrières', 'NOM'),
 ('MWE_STOP', 'NOM'),
 ('menu', 'ADJ'),
 ('sub', 'NOM'),
 ('haut', 'ADJ'),
 ('répertoire', 'NOM'),
 ('MWE_STOP', 'NAM')]

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [456]:
#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
#lemmatizer = FrenchLefffLemmatizer()

#lemmas = []
#for term in tagged:
#    term_l = []
#    if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    # elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
    #     term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    # else:
    #     term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    # lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [457]:
from nltk import everygrams
ngrammes = list(everygrams(tagged, min_len=2, max_len=6))
# ngrammes_lemmatized = list(everygrams(lemmas, min_len=2, max_len=5))

In [458]:
print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))

Avant filtrage, on a 4198775 ngrammes.


### **Extraction des patrons syntaxiques**

In [459]:
def extract_patterns(ngrammes):
    patterns = []

    for ng in ngrammes:
        phrase = [t[0] for t in ng]
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
        
    return patterns

In [460]:
phrases = extract_patterns(ngrammes)
# phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [461]:
# On calcule la distribution de fréquences de nos n-grammes
from nltk.probability import FreqDist

freq = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])
# freq_lemmatized = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases_lemmatized])

In [462]:
freq.most_common(25)

[('MWE_STOP MWE_STOP', 10478),
 ('du chum', 8694),
 ('de la', 7539),
 ('de recherche', 6335),
 ('centre de', 5331),
 ("à l'", 5162),
 ('le chum', 5055),
 ('chum MWE_STOP', 4846),
 ('MWE_STOP MWE_STOP MWE_STOP', 4675),
 ("de l'", 4210),
 ('centre de recherche', 4044),
 ('de montréal', 3619),
 ('université de', 3437),
 ('université de montréal', 3297),
 ("l'université", 3063),
 ("l'université de", 2952),
 ("l'université de montréal", 2831),
 ('MWE_STOP patients', 2742),
 ("à l'université", 2521),
 ("à l'université de", 2480),
 ("à l'université de montréal", 2418),
 ('MWE_STOP le', 2410),
 ('chum est', 2410),
 ('du réseau', 2408),
 ('enseignement et', 2365)]

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [463]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

In [464]:
def filtrer_stopwords(x):
    return [term for term in x if not 'MWE_STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords]

In [465]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un chiffre

In [466]:
def filter_num(x):
    return [term for term in x if not term[0][0].isnumeric() and not term[0][-1].isnumeric()]

In [467]:
phrases = filter_num(phrases)

On retire les n-grammes qui débutent ou se terminent par token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [468]:
def filter_len(x):
    return [term for term in x if \
        len(term[0][0]) > 2 and len(term[0][0]) < 18 and \
        len(term[0][-1]) > 2 and len(term[0][-1]) < 18]

In [469]:
phrases = filter_len(phrases)

In [470]:
phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [471]:
for phrase in phrases:
    phrase.append(freq[phrase[0]])

In [472]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 808011 occurrences de ngrammes.


In [473]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

,Expression,Patron syntaxique,Fréquence
0,patients répertoire,ADJ NOM,2298
1,patients répertoire enseignement,ADJ NOM NOM,2295
2,patients répertoire enseignement et académie,ADJ NOM NOM KON NOM,2295
3,patients répertoire enseignement et académie c...,ADJ NOM NOM KON NOM NOM,2295
4,répertoire enseignement,NOM NOM,2295
...,...,...,...
808006,regroupement des retraités du chum,NOM PRP:det NOM PRP:det NOM,2297
808007,regroupement des retraités du chum ruisss,NOM PRP:det NOM PRP:det NOM ADJ,2295
808008,retraités du chum,NOM PRP:det NOM,2299
808009,retraités du chum ruisss,NOM PRP:det NOM ADJ,2295


In [474]:
def tabCSV(phrases, titre):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    file_path = path.join(base_path, acteur, acteur)
    if sous_corpus:
       file_path = path.join(base_path, acteur, tag, tag)
    

    Path(file_path).mkdir(parents=True, exist_ok=True)

    tab.to_csv(file_path + titre)

    return tab.values.tolist()

In [475]:
phrases = tabCSV(phrases,'_n-grams.csv')
# phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [476]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 383246 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [477]:
file_path = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # # On peut aussi seulement prendree les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [478]:
terms = [t for t in phrases if t[1] in patterns and not 'NOM NOM' in t[1]] # 
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [479]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 60 % des termes
On avait 383246 ngrammes, on en a maintenant 153593.


In [480]:
import pandas as pd

def extract_terms(liste_terms, titre):
    file_path = '../04-filtrage/output/'
    tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    if sous_corpus:
        file_path = path.join(file_path, acteur, tag, tag)

    else :
        file_path = path.join(file_path, acteur, acteur)

                    
    tab.to_csv(file_path + titre)

extract_terms(terms, '_terms.csv')
#extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

In [481]:
terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])

In [482]:
terms_patterns = terms_patterns.to_dict('records')

In [483]:
dict_patterns = {}
for term in terms_patterns:
    exp = term['Expression']
    pattern = term['Structure syntaxique']
    dict_patterns[exp] = pattern

# dict_patterns

In [484]:
DataFrame(terms_patterns)

,Expression,Structure syntaxique,Fréquence
0,centre de recherche,VER:pres PRP NOM,4044
1,centre de recherche,NOM PRP NOM,4044
2,fondation du chum,NOM PRP:det NOM,2352
3,enseignement et académie,NOM KON NOM,2317
4,comité des usagers,NOM PRP:det NOM,2313
...,...,...,...
153588,immunologie en épidémiologie,NOM PRP NOM,1
153589,virologie en immunologie en épidémiologie,NOM PRP NOM PRP NOM,1
153590,domaines pertinents,NOM ADJ,1
153591,science dans plusieurs domaines,NOM PRP PRO:IND NOM,1


### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [485]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [486]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 153593 ngrammes.


In [487]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2
from nltk import bigrams, trigrams, ngrams

In [488]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [489]:
def llr_ngrammes(n):
    llr = []

    for i in range(2, n+1):
        ngrammes = set([tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == i])
        fd = nltk.FreqDist(ngrams(tokens, n=i))
        fd_prior = nltk.FreqDist(ngrams(tokens, n=i-1))
        
        for t in ngrammes:
            c_prior = fd_prior[t[:i-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[i-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = fd[t]             # Le ngramme lui-même P(w1w2w3..wn)

            res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
            p = chi2.sf(res, 1) # 1 degrees of freedom
            #if res == float('-inf') :
            #    res = 50000

            if p < 0.05 or (res == float('-inf')):
                llr.append({'Collocation' : " ".join(t).replace("' ", "'"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

    return llr

In [490]:
terms = llr_ngrammes(6)

In [ ]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len_prior - len(terms)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len_prior - len(terms)) / len_prior *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 12 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 1 % de nos n-grammes.


In [ ]:
DataFrame(terms).sort_values(by = "Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
541,centre de recherche,NOM PRP NOM,34,258.960385,2.891539e-58
446,fondation du chum,NOM PRP:det NOM,23,164.166818,1.390975e-37
420,enseignement et académie,NOM KON NOM,23,297.851259,9.680887e-67
1289,regroupement des retraités du chum,NOM PRP:det NOM PRP:det NOM,22,156.911497,5.351667e-36
1237,commissaire local aux plaintes politique,NOM ADJ PRP:det NOM ADJ,22,294.812807,4.445477e-66
...,...,...,...,...,...
550,inscription l'objectif,NOM DET:ART NOM,1,19.585858,9.617847e-06
549,bout du drain,NOM PRP:det NOM,1,14.581834,1.342020e-04
548,démontrer l'impact,VER:infi DET:ART NOM,1,16.813270,4.124389e-05
547,hassan falcone émilia,ADJ NOM VER:simp,1,19.585858,9.617847e-06


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; en tokenisant, on supprimer les frontières entre les mots qui sont des tirets ou autres caractères qu'un espace ou un apostrophe ; ça fait en sorte qu'on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [ ]:
dfs = {term['Collocation']: len([doc for doc in text if term['Collocation'] in doc]) for term in terms}

In [ ]:
test = {term for term in dfs if dfs[term] == 0}

In [ ]:
test

{'action 2018 medteq',
 'chum groupe voyages',
 'commission cv disease',
 'commission cv disease across the lifespan',
 'innove action 2018 medteq',
 'lancet commission cv disease',
 'speaker lancet commission cv disease'}

In [ ]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans 98% des documents 
terms = [t for t in terms if dfs[t['Collocation']] < max_df and dfs[t['Collocation']] > 0] 

In [ ]:
#for term in terms:
#    term['DF'] = dfs[term['Collocation']] 
DataFrame(terms).sort_values(by="Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
808,centre de prélèvement,VER:pres PRP NOM,15,151.428038,8.449563e-35
1140,retrouver dans l'hôpital,VER:infi PRP DET:ART NOM,13,176.123545,3.402966e-40
1126,chum pour vous retrouver,NOM PRP PRO:PER VER:infi,13,187.903779,9.118087e-43
815,plan du chum,NOM PRP:det NOM,13,92.106552,8.213297e-22
693,urgence le chum,NOM DET:ART NOM,13,71.120179,3.361147e-17
...,...,...,...,...,...
532,voyages au profit,NOM PRP:det NOM,1,19.585858,9.617847e-06
531,transplantation une occasion,NOM DET:ART NOM,1,16.813270,4.124389e-05
530,travaux ont permis,NOM VER:pres VER:pper,1,19.585858,9.617847e-06
529,expérimentation par excellence,NOM PRP NOM,1,15.087177,1.026583e-04


In [ ]:
df = pd.DataFrame(terms)
df.sort_values(['Fréquence'], 
            axis=0,
            ascending=[False], 
            inplace=True)

output_path = path.join('../04-filtrage/output/', acteur, acteur + '_significant-collocations.csv') 
df.to_csv(output_path)

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [ ]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)


extrant.to_csv(file_path + '_KWIC' +'.csv')

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize([t['Collocation'] for t in terms])

# **MODIF**

In [ ]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

termes_mesh[:15]

for t in termes_mesh:
    print(t, corpus.count(t))

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')